# Rate of transition between items that follow presentation of the same item
They considered transitions between items following a shared repeated item. They calculated the proportion of those items recalled in $S_j = {j + 1, j + 2}$ of which CMR then recalled an item in the set $S_i = {i + 1, i + 2}$. They also calculated the proportion of recalls $S_i$ of which CMR then transitioned to an item in the set $S_j$. They calculated the proportion of transitions for each of lags $j - i >= 4$, and represented the mean percent of transitions across these lags. 

To estimate the proportion of transitions that CMR would make at these lags in the absence of repeated items, they considered transitions in control lists matched to the same serial positions considered in the mixed lists. They matched these serial positions to 100 random shuffles of the control lists, and took the mean across the reshuffled datasets.



## Data Preparation

In [ ]:
from compmemlearn.datasets import prepare_lohnas2014_data

trials, events, list_length, presentations, list_types, rep_data, subjects = prepare_lohnas2014_data(
    '../../data/repFR.mat')

events.head()

,subject,list,item,input,output,study,recall,repeat,intrusion,condition
0,1,1,0,1,1.0,True,True,0,False,4
1,1,1,1,2,2.0,True,True,0,False,4
2,1,1,2,3,3.0,True,True,0,False,4
3,1,1,3,4,4.0,True,True,0,False,4
4,1,1,4,5,5.0,True,True,0,False,4


## Functions

In [ ]:
def shared_contiguity(trials, presentations, max_repeats=2, min_lag=4):
    
    list_length = len(presentations[0])
    total_transitions = 0
    complementary_transitions = 0
    terminus = np.sum(trials != 0, axis=1) - 1 # number of recalls per trial
    
    for trial_index in range(len(trials)):
        
        item_count = np.max(presentations[trial_index]) + 1
        si = np.zeros((item_count, 2), dtype=int)
        sj = np.zeros((item_count, 2), dtype=int)
        
        # we track possible positions using presentations and alt_presentations
        for item in range(item_count):
            pos = np.nonzero(presentations[trial_index] == item)[0]
            
            # then we track Si and Sj by identifying repeated items and storing item indices
            if (len(pos) > 1) and (pos[1] - pos[0] >= min_lag):
                
                assert np.all(pos < list_length - 2) # bounds issues don't affect result in this case - might later
                si[item, :] = presentations[trial_index, pos[0]+1:pos[0]+3] + 1
                sj[item, :] = presentations[trial_index, pos[1]+1:pos[1]+3] + 1
            
        # i can track at each recall index whether si or sj was seen, 
        for recall_index in range(terminus[trial_index]):
            
            current_item = presentations[trial_index][trials[trial_index, recall_index]-1] + 1
            next_item = presentations[trial_index][trials[trial_index, recall_index+1]-1] + 1
            
            # then peek at whether the next item was in complementary set, 
            # aggregating totals with each comparison
            for si_match in np.nonzero(si == current_item)[0]:
                complementary_transitions += next_item in sj[si_match]
            for sj_match in np.nonzero(sj == current_item)[0]:
                complementary_transitions += next_item in si[sj_match]
 
            total_transitions += 1
            
    return complementary_transitions/total_transitions

It looks like there are two remaining things to clarify:

First I have to figure out whether and how to track whether it's possible to make a transition to si or sj.

Next I have to figure out how often to increment total transitions.

In [ ]:
shared_contiguity(trials[list_types == 4], presentations[list_types == 4], 2, 4)

0.0391566265060241